# Verificar o ambiente

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct  6 01:19:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# Instalando bibliotecas e modelos necessários:

In [2]:
# Cria a pasta para o modelo
!mkdir -p modelo

# Baixa o modelo RealESRGAN_x4plus para a pasta criada
!wget -q https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P modelo

In [9]:
# Logo
!mkdir -p img
!wget -q https://github.com/Hevenicio/CIIA/blob/main/03%20-%20Restaurador%20de%20Imagens%20com%20IA/img/logo_CIIA.png -P img

In [4]:
!ls
!ls /content/modelo

img  modelo  sample_data
RealESRGAN_x4plus.pth


In [5]:
%%writefile requirements.txt

numpy
opencv-python
Pillow
torch>=1.7
torchvision>=0.8.0
tqdm
huggingface-hub
basicsr
facexlib
gfpgan
ffmpeg-python
streamlit
pyngrok
py-real-esrgan
huggingface-hub==0.20.3
streamlit-image-comparison

Writing requirements.txt


In [13]:
!pip install -q -r requirements.txt

# Criando o Scrip principal



---



In [12]:
%%writefile app.py

from streamlit_image_comparison import image_comparison
from py_real_esrgan.model import RealESRGAN
import streamlit as st
from PIL import Image
from io import BytesIO
import torch

# --- CSS Simples para Estilizar o Uploader ---
st.markdown("""
    <style>
        .st-emotion-cache-1jicfl2 {
            border: 2px dashed #a8a8a8;
            padding: 2rem;
            border-radius: 0.5rem;
            text-align: center;
        }
        .st-emotion-cache-1jicfl2:hover {
            border-color: #f8f9fa;
        }
    </style>
""", unsafe_allow_html = True)

# --- Funções de Processamento com Cache ---
@st.cache_resource
def carregar_modelo():
    """
    Carrega o modelo Real-ESRGAN. A função é armazenada em cache para
    que o modelo não seja recarregado a cada interação do usuário.
    """
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = RealESRGAN(device, scale = 4)
        model_path = 'modelo/RealESRGAN_x4plus.pth'
        model.load_weights(model_path)
        return model
    except FileNotFoundError:
        st.error("Arquivo do modelo não encontrado! Certifique-se de que 'RealESRGAN_x4plus.pth' está na pasta 'modelo'.")
        return None
    except Exception as e:
        st.error(f"Ocorreu um erro ao carregar o modelo: {e}")
        return None

def restaurar_imagem(imagem_pil, model):
    """
    Recebe uma imagem (formato PIL), a aprimora usando o modelo e a retorna.
    """
    if model is None:
        return None
    try:
        sr_image = model.predict(imagem_pil)
        return sr_image
    except Exception as e:
        st.error(f"Ocorreu um erro durante o processamento da imagem: {e}")
        return None

# --- Configuração da Página ---
st.set_page_config(layout = 'centered', page_icon = '✨🖼️', page_title = 'Restaurador de Imagens com IA')

# --- Interface Principal ---
st.markdown("<h2 style='text-align: center;'>Restaurador de Imagens com IA ✨🖼️</h2>", unsafe_allow_html = True)
st.markdown("<h5 style='text-align: center;'>Dê vida nova a fotos antigas, borradas ou de baixa qualidade.</h5>", unsafe_allow_html = True)
st.markdown("---")


# Carrega o modelo de IA
model = carregar_modelo()

with st.sidebar:
    #st.logo('img/logo_CIIA.png', size = 'large', )
    st.image('img/logo_CIIA.png', use_container_width = True)

    st.markdown('# 📤 Envie uma imagem')
    #uploaded_file = st.file_uploader('-', type = ['jpg', 'jpeg', 'png'],  help = 'Formatos suportados: JPG, JPEG, PNG', label_visibility = 'collapsed')
    uploaded_file = st.file_uploader('Selecione uma imagem para restaurar', type = ['png', 'jpeg', 'jpg'], label_visibility = 'collapsed')


if uploaded_file is not None and model is not None:
    if 'file_id' not in st.session_state or st.session_state.file_id != uploaded_file.file_id:
        st.session_state.file_id = uploaded_file.file_id
        st.session_state.restored_image = None

    imagem_original = Image.open(uploaded_file).convert('RGB')

    # Botão de restauração fica visível o tempo todo
    if st.sidebar.button('✨ Aplicar Restauração', width = 'content', type = 'primary'):
        with st.spinner('A image está sendo aprimorada... Por favor, aguarde.'):
            imagem_restaurada = restaurar_imagem(imagem_original, model)
            if imagem_restaurada:
                st.session_state.restored_image = imagem_restaurada

    # Lógica de exibição condicional
    if st.session_state.restored_image is None:
        # Se AINDA NÃO restaurou, mostra só a original
        st.sidebar.image(imagem_original, width = 'content', use_container_width = True)
    else:
        image_comparison(
            img1 = imagem_original,
            img2 = st.session_state.restored_image,
            label1 = 'Original',
            label2 = 'Restaurada',
            width = 700,
            starting_position = 50,
            show_labels = True,
            make_responsive = True,
            in_memory = True,
        )

        st.divider()

        buf = BytesIO()
        st.session_state.restored_image.save(buf, format = "PNG")
        byte_im = buf.getvalue()

        nome_base = uploaded_file.name.rsplit('.', 1)[0]
        nome_saida = f'{nome_base}_restaurada.png'

        st.sidebar.write('###### **Faça o download da imagem restaurada:**')
        st.sidebar.download_button(
            label = "Baixar em PNG",
            data = byte_im,
            file_name = nome_saida,
            mime = "image/png",
            width = 'content'
        )

Overwriting app.py




---



# Via NGROK
# Este código irá parar processos antigos, iniciar o app e mostrar a URL correta:

In [8]:
# 1. Para qualquer processo anterior do Streamlit ou ngrok para evitar erros
!kill -9 $(lsof -t -i:8501) &> /dev/null
!pkill ngrok &> /dev/null

from google.colab import userdata
from pyngrok import ngrok
import time
import os

# 2. Verifica se o app.py existe
if not os.path.exists('app.py'):
    print("❌ Erro: O arquivo 'app.py' não foi encontrado.")
    print("Por favor, execute a célula que cria o '%%writefile app.py' primeiro.")
else:
    # 3. Pega a chave do ngrok dos Secrets do Colab
    try:
        ngrok_key = userdata.get('key_ngrok')
        if not ngrok_key:
            raise ValueError("A chave 'key_ngrok' está vazia ou não foi encontrada.")

        print("🔑 Chave do ngrok encontrada. Autenticando...\n")
        ngrok.set_auth_token(ngrok_key)

        # 4. Inicia o servidor Streamlit em background
        print("🚀 Iniciando o servidor Streamlit em segundo plano...\n")
        os.system("streamlit run app.py &")

        # Pequena pausa para garantir que o servidor subiu
        time.sleep(5)

        # 5. Conecta o ngrok à porta 8501 e obtém a URL pública
        print("🔗 Criando o túnel público com ngrok...")
        public_url = ngrok.connect(8501)

        print("\n\n✅ TUDO PRONTO! ✅")
        print("👇👇👇 Clique no link abaixo para abrir seu aplicativo 👇👇👇\n")
        print(public_url)

    except Exception as e:
        print(f"\n❌ Ocorreu um erro: {e}")
        print("Por favor, verifique se sua chave 'key_ngrok' está correta nos Secrets do Colab.")

🔑 Chave do ngrok encontrada. Autenticando...

🚀 Iniciando o servidor Streamlit em segundo plano...

🔗 Criando o túnel público com ngrok...


✅ TUDO PRONTO! ✅
👇👇👇 Clique no link abaixo para abrir seu aplicativo 👇👇👇

NgrokTunnel: "https://thermodynamic-cortez-palaeobotanical.ngrok-free.dev" -> "http://localhost:8501"
